In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
import tensorflow as tf
import numpy as np
slim=tf.contrib.slim

from nsynth_finetuned_encoder import *
from librispeech_reader import *

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [3]:
CHECKPOINT = "/workspace/models_pretrained/nsynth_finetuned_encoder/model.ckpt-56614"
# CHECKPOINT = "/workspace/models_pretrained/nsynth_finetuned_encoder/"
LOGDIR = "/workspace/models_pretrained/nsynth_finetuned_encoder_eval/"
EVAL_PATH = '/workspace/data/LibriSpeech_to_classify/val/wavs.tfrecord'

In [4]:
def get_batch(batch_size, path = ''):
    print('Get batch')
    assert path is not None
    data = librispeech_reader.LibriSpeechDataset(
        path, is_training = True)
    return data.get_wavenet_batch(batch_size, length = 39936)

In [5]:
def evaluate_classifier(inputs_dict, num_evals=1):
    labels = inputs_dict['label']
    
    model = Config(train_path = EVAL_PATH)
    outputs_dict = model.build(inputs_dict, is_training=False)
    
    
    logits = outputs_dict['predictions_speaker']
    predictions = tf.argmax(logits, 1, output_type = tf.int32)
    
    # Define the metrics:
    names_to_values, names_to_updates = slim.metrics.aggregate_metric_map({
        'eval/Accuracy': tf.metrics.accuracy(labels, predictions),
    })

    print('Running evaluation Loop...')
    checkpoint_path = CHECKPOINT 
    logdir = LOGDIR

    metric_values = slim.evaluation.evaluate_once(
        num_evals=num_evals,
        master='',
        checkpoint_path=checkpoint_path,
        logdir=logdir,
        eval_op=list(names_to_updates.values()),
        final_op=list(names_to_values.values()))

    names_to_values = dict(zip(names_to_values.keys(), metric_values))
    for name in names_to_values:
        print('%s: %f' % (name, names_to_values[name]))


In [12]:
tf.logging.set_verbosity(tf.logging.INFO)

for i in range(10):
    graph = tf.Graph()
    with graph.as_default():
        global_step = tf.get_variable(
            "global_step", [],
            tf.int32,
            initializer=tf.constant_initializer(0),
            trainable=False)

        cpu_device = "/job:localhost/replica:0/task:0/device:CPU:0"

        with tf.device(cpu_device):
            inputs_dict = get_batch(120, EVAL_PATH)

        evaluate_classifier(inputs_dict, num_evals=2)


Get batch
Running evaluation Loop...
INFO:tensorflow:Starting evaluation at 2018-05-23-20:07:17
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /workspace/models_pretrained/nsynth_finetuned_encoder/model.ckpt-56614
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/2]
INFO:tensorflow:Evaluation [2/2]
INFO:tensorflow:Finished evaluation at 2018-05-23-20:07:32
eval/Accuracy: 0.850000
Get batch
Running evaluation Loop...
INFO:tensorflow:Starting evaluation at 2018-05-23-20:07:34
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /workspace/models_pretrained/nsynth_finetuned_encoder/model.ckpt-56614
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/2]
INFO:tensorflow:Evaluation [2/2]
INFO:tensorflow:Finished evaluation at 2018-05-23-20:07:49
eval/Accuracy: 0.904167
Get batch
Running evaluation Loop...
INFO:te